In [2]:
import os
import glob
import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import pandas as pd

import MetaTrader5 as mt5
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

import config

In [3]:
account = 51913983
password = "drbj2iln"
server = "Alpari-MT5-Demo"

In [22]:
mt5.initialize()
authorized=mt5.login(account, password=password, server=server)
pd. set_option('display.max_columns', None) 

In [23]:
account_info_dict = mt5.account_info()._asdict()
account_info_df = pd.DataFrame(account_info_dict, index=[0])

In [24]:
account_info_df

,login,trade_mode,leverage,limit_orders,margin_so_mode,trade_allowed,trade_expert,margin_mode,currency_digits,fifo_close,balance,credit,profit,equity,margin,margin_free,margin_level,margin_so_call,margin_so_so,margin_initial,margin_maintenance,assets,liabilities,commission_blocked,name,server,currency,company
0,51913983,0,100,10000,0,True,True,2,2,False,9880.58,0.0,0.0,9880.58,0.0,9880.58,0.0,80.0,60.0,0.0,0.0,0.0,0.0,0.0,vatche thorossian,Alpari-MT5-Demo,USD,Alpari


In [6]:
print("Profit:", account_info_df["profit"].iloc[0])
print("Equity:", account_info_df["equity"].iloc[0])
print("Margin:", account_info_df["margin"].iloc[0])
print("Margin Free:", account_info_df["margin_free"].iloc[0])

Profit: 0.0
Equity: 9880.58
Margin: 0.0
Margin Free: 9880.58


In [7]:
from_date=datetime(2020,1,1)
to_date=datetime.now()
history_data=(mt5.history_orders_get(from_date, to_date))

In [20]:
trade_data_dataframe=pd.DataFrame(list(history_data),columns=history_data[0]._asdict().keys())


In [21]:
trade_data_dataframe.head()

,ticket,time_setup,time_setup_msc,time_done,time_done_msc,time_expiration,type,type_time,type_filling,state,magic,position_id,position_by_id,reason,volume_initial,volume_current,price_open,sl,tp,price_current,price_stoplimit,symbol,comment,external_id
0,189649925,1701343191,1701343191743,1701343191,1701343191789,0,1,0,0,4,0,189649925,0,0,1.0,0.0,0.0,0.0,0.0,147.076,0.0,USDJPY,,
1,189649931,1701343193,1701343193481,1701343193,1701343193536,0,0,0,0,4,0,189649931,0,0,1.0,0.0,0.0,0.0,0.0,147.082,0.0,USDJPY,,
2,189649933,1701343194,1701343194186,1701343194,1701343194260,0,1,0,0,4,0,189649933,0,0,1.0,0.0,0.0,0.0,0.0,147.075,0.0,USDJPY,,
3,189649934,1701343194,1701343194863,1701343194,1701343194914,0,1,0,0,4,0,189649934,0,0,1.0,0.0,0.0,0.0,0.0,147.075,0.0,USDJPY,,
4,189649985,1701343212,1701343212008,1701343212,1701343212067,0,0,0,0,4,0,189649985,0,0,1.0,0.0,0.0,0.0,0.0,147.088,0.0,USDJPY,,


In [11]:
from_date=datetime(2020,1,1)
to_date=datetime.now()
history_deals=(mt5.history_deals_get(from_date, to_date))

In [12]:
deals_dataframe=pd.DataFrame(list(history_deals),columns=history_deals[0]._asdict().keys())

In [25]:
deals_dataframe.head()

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id
0,156509592,0,1701343107,1701343107010,2,0,0,0,0,0.0,0.000,0.0,0.0,10000.0,0.0,,,
1,156509735,189649925,1701343191,1701343191789,1,0,0,189649925,0,1.0,147.076,-1.6,0.0,0.0,0.0,USDJPY,,
2,156509740,189649931,1701343193,1701343193536,0,0,0,189649931,0,1.0,147.082,-1.6,0.0,0.0,0.0,USDJPY,,
3,156509742,189649933,1701343194,1701343194260,1,0,0,189649933,0,1.0,147.075,-1.6,0.0,0.0,0.0,USDJPY,,
4,156509743,189649934,1701343194,1701343194914,1,0,0,189649934,0,1.0,147.075,-1.6,0.0,0.0,0.0,USDJPY,,


In [27]:

deals_dataframe['time'] = pd.to_datetime(deals_dataframe['time'], unit='s')
# Extracting relevant information from 'deals_dataframe'
start_date = deals_dataframe['time'].iloc[0]
number_of_trades = len(deals_dataframe)
win_rate = (deals_dataframe['profit'] > 0).sum() / number_of_trades if number_of_trades > 0 else 0
average_win = deals_dataframe[deals_dataframe['profit'] > 0]['profit'].mean()
average_loss = deals_dataframe[deals_dataframe['profit'] < 0]['profit'].mean()

# Calculating additional metrics
account_size = account_info_df['balance'].iloc[0]
equity = account_info_df['equity'].iloc[0]
profit_target = 16000  # Set your desired profit target
daily_loss = 0  # Placeholder, you need to calculate this based on your data
max_loss = 0.1 * account_size  # Assuming the maximum loss is 10% of the initial account balance
active_trading_days = len(deals_dataframe['time'].dt.date.unique())

# Creating the final dataframe
final_data = pd.DataFrame({
    'Start Date': [start_date],
    'Account Size': [account_size],
    'Equity': [equity],
    'Status': ['Green'],  # Placeholder, you need to determine the status based on your criteria
    'Profit Target': [profit_target],
    'Daily Loss': [daily_loss],
    'Maximum Loss': [max_loss],
    'Active Trading Days': [active_trading_days],
    'Number of Trades': [number_of_trades],
    'Winrate': [win_rate],
    'Average Win': [average_win],
    'Average Loss': [average_loss],
    'Platform': ['MetaTrader 5'],  # You can change this based on your platform
    'Server': [account_info_df['server'].iloc[0]],
})

# Displaying the final dataframe
final_data

,Start Date,Account Size,Equity,Status,Profit Target,Daily Loss,Maximum Loss,Active Trading Days,Number of Trades,Winrate,Average Win,Average Loss,Platform,Server
0,2023-11-30 11:18:27,9880.58,9880.58,Green,16000,0,988.058,1,49,0.408163,532.727,-135.6,MetaTrader 5,Alpari-MT5-Demo
